In [83]:
import sys
sys.path.append('/')
import torch
import torch.nn as nn
import numpy as np

max_prop = 1000
coaxialeff = 450000/(max_prop**2)
coefthrust = 5.7199 / (max_prop**2)
coeftorque = 0.4008 / (max_prop**2)
k = coefthrust
b = coeftorque
Lx = 0.36 / 2
Ly = 0.26 / 2 
Lz = 0.0221

BATCH = 256

alpha = torch.randn(size=[BATCH, 1]).cuda()
d = torch.randn(size=[BATCH, 1]).cuda()
controlmatrix = np.zeros([BATCH, 4, 8])
d_scale = 0.43/2.0
dcg = d * d_scale + d_scale
c = torch.cos(alpha*torch.pi/2.0)
s = torch.sin(alpha*torch.pi/2.0)

In [84]:
c

tensor([[ 0.6795],
        [ 0.5437],
        [ 0.7230],
        [ 0.9860],
        [ 0.3331],
        [ 0.6422],
        [ 0.9298],
        [-0.2829],
        [ 0.4912],
        [ 0.6392],
        [ 0.7430],
        [ 0.9822],
        [ 0.7229],
        [ 0.9226],
        [ 0.3674],
        [-0.3948],
        [ 0.8354],
        [ 1.0000],
        [-0.6449],
        [ 0.4644],
        [ 0.9789],
        [ 0.1249],
        [ 0.0642],
        [ 0.8616],
        [ 0.7673],
        [-0.9410],
        [-0.3949],
        [ 0.4526],
        [ 0.8878],
        [-0.4240],
        [ 0.8504],
        [ 0.1057],
        [ 0.9974],
        [-0.5917],
        [-0.6508],
        [ 0.5900],
        [ 0.2424],
        [ 0.5310],
        [-0.4113],
        [ 0.9963],
        [ 0.9784],
        [ 0.7600],
        [ 0.5439],
        [-0.9870],
        [ 0.9509],
        [ 0.9583],
        [-0.5072],
        [-0.7406],
        [ 0.0957],
        [ 0.7396],
        [-0.4920],
        [ 0.2898],
        [ 0.

In [85]:
matrixfront = torch.Tensor([[Lx*k, Lx*k, -Lx*k, -Lx*k], 
                         [-Ly*k, -Ly*k, -Ly*k, -Ly*k], 
                         [b, -b, -b, b], 
                         [k, k, k, k]]).type(torch.float32).cuda()
matrixfront = matrixfront.unsqueeze(dim=0).expand(BATCH, 4, 4)
d_bias = torch.Tensor([[0,0,0,0], 
                         [1,1,1,1], 
                         [0,0,0,0], 
                         [0,0,0,0]]).type(torch.float32).cuda()
d_bias = d_bias.unsqueeze(dim=0).expand(BATCH, 4, 4)
d_bias = d_bias * dcg.unsqueeze(dim=1).expand(BATCH, 4, 4)
matrixfront = matrixfront + d_bias * k

In [86]:
dcg[0]

tensor([-0.2346], device='cuda:0')

In [87]:
d_bias[0,:,:]

tensor([[-0.0000, -0.0000, -0.0000, -0.0000],
        [-0.2346, -0.2346, -0.2346, -0.2346],
        [-0.0000, -0.0000, -0.0000, -0.0000],
        [-0.0000, -0.0000, -0.0000, -0.0000]], device='cuda:0')

In [88]:
matrixback = torch.Tensor([[-Lx*k, -Lx*k, Lx*k, Lx*k], 
                         [Ly*k, Ly*k, Ly*k, Ly*k], 
                         [b, -b, -b, b], 
                         [k, k, k, k]]).type(torch.float32).cuda()
matrixback = matrixback.unsqueeze(dim=0).expand(BATCH, 4, 4)
matrixback = matrixback * c.unsqueeze(dim=1).expand(BATCH, 4, 4)

d_bias = torch.Tensor([[0,0,0,0], 
                         [1,1,1,1], 
                         [0,0,0,0], 
                         [0,0,0,0]]).type(torch.float32).cuda()
d_bias = d_bias.unsqueeze(dim=0).expand(BATCH, 4, 4)
d_bias = d_bias * dcg.unsqueeze(dim=1).expand(BATCH, 4, 4)
matrixback = matrixback + d_bias * k * c.unsqueeze(dim=1).expand(BATCH, 4, 4)

bias_roll = torch.Tensor([[-1,-1,-1,-1], 
                         [0,0,0,0], 
                         [0,0,0,0], 
                         [0,0,0,0]]).type(torch.float32).cuda()
bias_roll = bias_roll.unsqueeze(dim=0).expand(BATCH, 4, 4)
bias_roll = bias_roll * s.unsqueeze(dim=1).expand(BATCH, 4, 4) * b
matrixback = matrixback + bias_roll

bias_z = torch.Tensor([[0,0,0,0], 
                    [-1,-1,-1,-1], 
                    [0,0,0,0], 
                    [0,0,0,0]]).type(torch.float32).cuda()
bias_z = bias_z.unsqueeze(dim=0).expand(BATCH, 4, 4)
bias_z = bias_z * s.unsqueeze(dim=1).expand(BATCH, 4, 4) * Lz * k
matrixback = matrixback + bias_z

controlmatrix = torch.concat([matrixfront, matrixback], dim=2)

In [89]:
controlmatrix.shape

torch.Size([256, 4, 8])

In [90]:
2 ** 15

32768